In [1]:
import numpy as np
import emcee as em
import matplotlib.pyplot as plt
from ipywidgets import interact
import keras as ks
import emcee

Using TensorFlow backend.


In [2]:
%ls

 98bk02.h5         dTb02              index.png   'paramter estimation.ipynb'
 binary_data_02/   EmuBk0.2.h5        lvac02.png   x_HI02
 bk_norm02         hyperparam.ipynb   nbins02
 bk_un02           index              params02


In [3]:
N_bins = np.loadtxt('nbins02')
N_bins.shape

(1058, 550)

In [4]:
Bk = np.loadtxt('bk_norm02')
Bk.shape

(1058, 550)

In [5]:
params = np.loadtxt('params02')
params.shape

(1058, 3)

In [6]:
sigma = Bk/np.sqrt(N_bins)
cov_inv = 1./sigma

/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


In [7]:
cov_inv.shape

(1058, 550)

In [8]:
cov = np.zeros(shape=(1058,550,550))
for i in range(1058):
    for j in range(550):
        cov[i][j][j] = cov_inv[i][j]

In [9]:
ind = np.loadtxt('index')
index = np.zeros(len(ind),dtype=int)
for i in range(len(ind)):
    index[i] = int(ind[i])
index

array([ 146,  833,  516,  204,  929,  223,  154,   42,   87,   10,  535,
        343,  525,  239,   37,  532,  966, 1043])

In [10]:
Bk_test = Bk[index]
params_test = params[index]
cov_inv_test = cov[index]

In [11]:
cov_inv_test[0].shape

(550, 550)

In [12]:
np.dot(np.dot(Bk_test[0],cov_inv_test[0]),Bk_test[0])

2455097.5502784983

In [13]:
def Bk_model(params):
    
    model = ks.models.load_model('98bk02.h5')
    model = model.predict([[params]])
    model = model*100.
    
    return model

In [14]:
def lL(params,Bk_test,cov_inv_test):
    Nion,Rmfp,NoH = params
    if Nion<=0. or Rmfp<=0. or NoH<=0. :
        l=-np.inf
    else:
        #covv = np.reshape(cov_inv_test[0],(1,550))
        model=Bk_model(params)
        DMU=Bk_test[0]-model
        l=(-0.5)*np.dot(np.dot(DMU,cov_inv_test[0]),DMU.T)
   
    return l

In [15]:
cov_inv_test[0].shape

(550, 550)

In [16]:
cov.shape

(1058, 550, 550)

In [17]:
ndim=3            #===== no. of parameters =======#

nwalkers=10      #========= no. of random walkers ========#

p=np.random.rand(nwalkers,ndim)   #======== intial values of parameters ========#
p[0]

array([0.49599819, 0.37056611, 0.3381715 ])

In [18]:
nion,rmfp,noh = p[0]
nion

0.4959981858872192

In [19]:
lL(p[0],Bk_test,cov_inv_test)

array([[-755877.84144064]])

In [20]:
sampler=emcee.EnsembleSampler(nwalkers,ndim,lL,args=[Bk_test,cov_inv_test])

In [21]:
pos, prob,stats=sampler.run_mcmc(p, 100,)
sampler.reset()

emcee: Exception while calling your likelihood function:
  params: 

KeyboardInterrupt: 

In [ ]:
pos

In [ ]:
N = 10000
sampler.run_mcmc(pos,N,store = True)

In [ ]:
import corner

In [ ]:
s_flat=sampler.flatchain

In [ ]:
emcee.__version__

In [ ]:
chains = sampler.chain